In [1]:
import os
os.chdir('../')
os.getcwd()
from old_type.TwoStageTrAdaBoostR2 import *
from old_type.TransferStacking import TransferStacking

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso

In [23]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [3]:
def make_dataset_concrete():
    data = arff.loadarff('./dataset/transfer data/new-concrete1.arff')
    df1 = pd.DataFrame(data[0])
    df1["label"] = 0
    data = arff.loadarff('./dataset/transfer data/new-concrete2.arff')
    df2 = pd.DataFrame(data[0])
    df2["label"] = 1
    data = arff.loadarff('./dataset/transfer data/new-concrete3.arff')
    df3 = pd.DataFrame(data[0])
    df3["label"] = 2
    df = pd.concat([df1, df2, df3], sort=False).reset_index(drop=True)
    return df

In [4]:
df = make_dataset_concrete()
df

,b,c,d,e,f,g,h,i,label
0,153.0,0.0,192.0,0.0,887.0,942.0,3.0,4.57,0
1,153.0,0.0,192.0,0.0,887.0,942.0,90.0,25.46,0
2,153.0,0.0,192.0,0.0,887.0,942.0,7.0,7.68,0
3,153.0,0.0,192.0,0.0,887.0,942.0,28.0,17.28,0
4,162.4,0.0,203.5,0.0,938.2,849.0,28.0,20.59,0
...,...,...,...,...,...,...,...,...,...
1025,0.0,0.0,173.0,0.0,1125.0,613.0,28.0,67.31,2
1026,0.0,0.0,173.0,0.0,1125.0,613.0,90.0,69.66,2
1027,0.0,0.0,173.0,0.0,1125.0,613.0,180.0,71.62,2
1028,0.0,0.0,173.0,0.0,1125.0,613.0,270.0,74.17,2


In [5]:
n_target=25
y_feature="i"

In [6]:
max_depth = 3
n_estimators = 30
steps =30
fold = 10
random_state = np.random.RandomState(1)
base_estimator = DecisionTreeRegressor(max_depth=max_depth)
meta_model = Lasso(alpha=0.1)

In [12]:
target_domain = 0

target_train = df.query('label==@target_domain').sample(n_target)
target_test = df.query('label==@target_domain').drop(index=target_train.index)
target_train_X = target_train.drop(columns=[y_feature, 'label'])
target_train_y = target_train[y_feature]
target_test_X = target_test.drop(columns=[y_feature, 'label']).values
target_test_y = target_test[y_feature].values
source_X = df.query('label!=@target_domain').drop(columns=[y_feature, 'label'])
source_y = df.query('label!=@target_domain')[y_feature]

X_train = pd.concat([source_X, target_train_X]).values
y_train = pd.concat([source_y, target_train_y]).values
sample_size = [source_X.shape[0], target_train_X.shape[0]]


# two-stage
# regr_tr = TwoStageTrAdaBoostR2(base_estimator,
#                             sample_size=sample_size, 
#                             n_estimators = n_estimators,
#                             steps = steps, fold = fold,
#                             random_state = random_state)
# regr_tr.fit(X_train, y_train)
# y_pred_tr = regr_tr.predict(target_test_X)
# rmse_tr = mean_squared_error(target_test_y, y_pred_tr)**0.5

# targetのみadaboost
regr_targetada = AdaBoostRegressor(base_estimator,
                    n_estimators = n_estimators, learning_rate=1.,
                    random_state = random_state)
regr_targetada.fit(target_train_X.values, target_train_y.values)
y_pred_targetada = regr_targetada.predict(target_test_X)
rmse_targetada = mean_squared_error(target_test_y, y_pred_targetada)**0.5

# source+target AdaBoost
regr_ada = AdaBoostRegressor(base_estimator,
                    n_estimators = n_estimators, learning_rate=1.,
                    random_state = random_state)
regr_ada.fit(X_train, y_train)
y_pred_ada = regr_ada.predict(target_test_X)
rmse_ada = mean_squared_error(target_test_y, y_pred_ada)**0.5

# Transfer Stacking
regr_src = AdaBoostRegressor(base_estimator,
                    n_estimators = n_estimators, learning_rate=1.,
                    random_state = random_state)
regr_src.fit(source_X.values, source_y.values)
regr_stacked = TransferStacking(regr_src, meta_model = Lasso(),
                                search_params = {"alpha":[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]},
                               fold=fold, random_state=random_state)
regr_stacked.fit(target_train_X.values, target_train_y.values)
y_pred_stacked = regr_stacked.predict(target_test_X)
rmse_stacked = mean_squared_error(target_test_y, y_pred_stacked)**0.5

print('adaboost target : {:.4}'.format(rmse_targetada))
print('adaboost all : {:.4}'.format(rmse_ada))
# print('two-stage TrAdaBoost.R2 : {:.4}'.format(rmse_tr))
print('Transfer Stacking : {:.4}'.format(rmse_stacked))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.058859064457031, tolerance: 0.51613418
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.4496946213460715, tolerance: 0.4281694527272728
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.029807268533901, tolerance: 0.5225083150000001
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 

adaboost target : 11.03
adaboost all : 15.42
Transfer Stacking : 11.11


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4447237423324815, tolerance: 0.47530536608695656
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8256850200594243, tolerance: 0.4899586191304348
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.340194182760932, tolerance: 0.51613418
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap

In [25]:
def experimence(df, target_domain, base_estimator):
    results = []
    for count in range(30):
        print(str(count).center(100,"-"))
        target_train = df.query('label==@target_domain').sample(n_target)
        target_test = df.query('label==@target_domain').drop(index=target_train.index)
        target_train_X = target_train.drop(columns=[y_feature, 'label'])
        target_train_y = target_train[y_feature]
        target_test_X = target_test.drop(columns=[y_feature, 'label']).values
        target_test_y = target_test[y_feature].values
        source_X = df.query('label!=@target_domain').drop(columns=[y_feature, 'label'])
        source_y = df.query('label!=@target_domain')[y_feature]

        X_train = pd.concat([source_X, target_train_X]).values
        y_train = pd.concat([source_y, target_train_y]).values
        sample_size = [source_X.shape[0], target_train_X.shape[0]]


        # two-stage
        regr_tr = TwoStageTrAdaBoostR2(base_estimator,
                                    sample_size=sample_size, 
                                    n_estimators = n_estimators,
                                    steps = steps, fold = fold,
                                    random_state = random_state)
        regr_tr.fit(X_train, y_train)
        y_pred_tr = regr_tr.predict(target_test_X)
        rmse_tr = mean_squared_error(target_test_y, y_pred_tr)**0.5

        # targetのみadaboost
        regr_targetada = AdaBoostRegressor(base_estimator,
                            n_estimators = n_estimators, learning_rate=1.,
                            random_state = random_state)
        regr_targetada.fit(target_train_X.values, target_train_y.values)
        y_pred_targetada = regr_targetada.predict(target_test_X)
        rmse_targetada = mean_squared_error(target_test_y, y_pred_targetada)**0.5

        # source+target AdaBoost
        regr_ada = AdaBoostRegressor(base_estimator,
                            n_estimators = n_estimators, learning_rate=1.,
                            random_state = random_state)
        regr_ada.fit(X_train, y_train)
        y_pred_ada = regr_ada.predict(target_test_X)
        rmse_ada = mean_squared_error(target_test_y, y_pred_ada)**0.5
        
        # Transfer Stacking
        regr_src = AdaBoostRegressor(base_estimator,
                            n_estimators = n_estimators, learning_rate=1.,
                            random_state = random_state)
        regr_src.fit(source_X.values, source_y.values)
        regr_stacked = TransferStacking(regr_src, meta_model = Lasso(),
                                        search_params = {"alpha":[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]},
                                       fold=fold, random_state=random_state)
        regr_stacked.fit(target_train_X.values, target_train_y.values)
        y_pred_stacked = regr_stacked.predict(target_test_X)
        rmse_stacked = mean_squared_error(target_test_y, y_pred_stacked)**0.5
        
        results.append([rmse_targetada, rmse_ada, rmse_stacked, rmse_tr])
    return pd.DataFrame(results, columns=["target", "all", "stacking", "two-stage"])
#         results.append([rmse_targetada, rmse_ada, rmse_stacked])
#     return pd.DataFrame(results, columns=["target", "all", "stacking"])

In [26]:
max_depth = 3

df_results_low = experimence(df, 0, base_estimator)
df_results_medium = experimence(df, 1, base_estimator)
df_results_high = experimence(df, 2, base_estimator)

results_dt = pd.concat([df_results_low.mean(), df_results_medium.mean(), df_results_high.mean()], axis=1).T
results_dt

-------------------------------------------------0--------------------------------------------------
-------------------------------------------------1--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
-------------------------------------------------2--------------------------------------------------
At step: 28
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
-------------------------------------------------3--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
------------

-------------------------------------------------4--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
-------------------------------------------------5--------------------------------------------------
-------------------------------------------------6--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
-------------------------------------------------7--------------------------------------------------
At step: 28
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
------------

-------------------------------------------------8--------------------------------------------------
-------------------------------------------------9--------------------------------------------------
At step: 28
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
-------------------------------------------------10-------------------------------------------------
-------------------------------------------------11-------------------------------------------------
-------------------------------------------------12-------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
-------------------------------------------------13------

,target,all,stacking,two-stage
0,9.834842,14.241433,12.380534,10.409802
1,10.597119,8.108072,11.935450,9.791939
2,11.456146,13.366149,16.490334,13.310961
